In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error

from src.constants import *
from src.utility import *
from src.avgs import *
from src.form import *

from src.rolling_data import RollingData
from src.per_game_model import PerGameModel
from src.init_DFs.per_game import PerGameInit
from src.init_DFs.next_game import NextGameInit

color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

pgModel = PerGameModel()
initPG = PerGameInit()
initNextGame = NextGameInit()

In [5]:
season = "all"
situation = "5on5"

rec_num = 4
past_num = 32

df = pd.read_csv(CSV_DB_PATH + f"{season}/NHL_5v5.csv")
pd.read_csv(CSV_DB_PATH + f"{season}/ROLLING_8_5v5.csv")


,game_id,team,opp_team,next_game_id,game_date,game_time,situation,season,game_number,score,...,scoreAdjustedShotsAttempts,unblockedShotAttempts,scoreAdjustedUnblockedShotAttempts,dZoneGiveaways,xGoalsFromxReboundsOfShots,xGoalsFromActualReboundsOfShots,reboundxGoals,totalShotCredit,scoreAdjustedTotalShotCredit,scoreFlurryAdjustedTotalShotCredit
0,2014-10-08BOSPHI,BOS,PHI,2014-10-09DETBOS,2014-10-08,19:30:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-10-08BOSPHI,PHI,BOS,2014-10-09PHINJD,2014-10-08,19:30:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-10-08CGYVAN,CGY,VAN,2014-10-09EDMCGY,2014-10-08,22:00:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-10-08CGYVAN,VAN,CGY,2014-10-11VANEDM,2014-10-08,22:00:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-10-08LAKSJS,LAK,SJS,2014-10-11ARILAK,2014-10-08,22:00:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21541,2023-04-13WSHNJD,WSH,NJD,NaN,2023-04-13,19:00:00,5on5,2022,82,2.375,...,41.246250,31.750,31.113375,5.625,0.301125,0.600375,0.600375,1.817750,1.785500,1.738375
21542,2023-04-14CBJBUF,BUF,CBJ,NaN,2023-04-14,19:30:00,5on5,2022,82,4.000,...,49.983750,39.125,40.003375,2.750,0.402375,0.705625,0.705625,2.520000,2.596625,2.541625
21543,2023-04-14CBJBUF,CBJ,BUF,NaN,2023-04-14,19:30:00,5on5,2022,82,1.875,...,38.372625,29.375,28.164250,3.375,0.303625,0.363000,0.363000,1.932000,1.857875,1.836750
21544,2023-04-14NSHCOL,COL,NSH,NaN,2023-04-14,20:00:00,5on5,2022,82,4.125,...,49.590625,33.625,35.673625,3.375,0.345500,0.477375,0.477375,2.186875,2.317250,2.279875


In [3]:
df_szn = get_season_avgs(df.copy(), False)
df_2 = get_rolling_avgs(df.copy(), 2, False, False)
df_4 = get_rolling_avgs(df.copy(), 4, False, False)
df_8 = get_rolling_avgs(df.copy(), 8, False, False)
df_16 = get_rolling_avgs(df.copy(), 16, False, False)
df_32 = get_rolling_avgs(df.copy(), 32, False, False)
df_8

,game_id,team,opp_team,next_game_id,game_date,game_time,situation,season,game_number,score,...,scoreAdjustedShotsAttempts,unblockedShotAttempts,scoreAdjustedUnblockedShotAttempts,dZoneGiveaways,xGoalsFromxReboundsOfShots,xGoalsFromActualReboundsOfShots,reboundxGoals,totalShotCredit,scoreAdjustedTotalShotCredit,scoreFlurryAdjustedTotalShotCredit
0,2014-10-08BOSPHI,BOS,PHI,2014-10-09DETBOS,2014-10-08,19:30:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-10-08BOSPHI,PHI,BOS,2014-10-09PHINJD,2014-10-08,19:30:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-10-08CGYVAN,CGY,VAN,2014-10-09EDMCGY,2014-10-08,22:00:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-10-08CGYVAN,VAN,CGY,2014-10-11VANEDM,2014-10-08,22:00:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-10-08LAKSJS,LAK,SJS,2014-10-11ARILAK,2014-10-08,22:00:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21541,2023-04-13WSHNJD,WSH,NJD,NaN,2023-04-13,19:00:00,5on5,2022,82,2.375,...,41.246250,31.750,31.113375,5.625,0.301125,0.600375,0.600375,1.817750,1.785500,1.738375
21542,2023-04-14CBJBUF,BUF,CBJ,NaN,2023-04-14,19:30:00,5on5,2022,82,4.000,...,49.983750,39.125,40.003375,2.750,0.402375,0.705625,0.705625,2.520000,2.596625,2.541625
21543,2023-04-14CBJBUF,CBJ,BUF,NaN,2023-04-14,19:30:00,5on5,2022,82,1.875,...,38.372625,29.375,28.164250,3.375,0.303625,0.363000,0.363000,1.932000,1.857875,1.836750
21544,2023-04-14NSHCOL,COL,NSH,NaN,2023-04-14,20:00:00,5on5,2022,82,4.125,...,49.590625,33.625,35.673625,3.375,0.345500,0.477375,0.477375,2.186875,2.317250,2.279875


In [4]:
df_szn.to_csv(CSV_DB_PATH + f'{season}/SEASON_AVG_5v5.csv', header=True, index=False)
df_2.to_csv(CSV_DB_PATH + f'{season}/ROLLING_2_5v5.csv', header=True, index=False)
df_4.to_csv(CSV_DB_PATH + f'{season}/ROLLING_4_5v5.csv', header=True, index=False)
df_8.to_csv(CSV_DB_PATH + f'{season}/ROLLING_8_5v5.csv', header=True, index=False)
df_16.to_csv(CSV_DB_PATH + f'{season}/ROLLING_16_5v5.csv', header=True, index=False)
df_32.to_csv(CSV_DB_PATH + f'{season}/ROLLING_32_5v5.csv', header=True, index=False)


In [10]:
df_2vSZN = get_form_quick(df_2.copy(), df_szn.copy(), 2, 2, True, ["season", "game_number"], False)
df_4vSZN = get_form_quick(df_4.copy(), df_szn.copy(), 2, 2, True, ["season", "game_number"], False)
df_8vSZN = get_form_quick(df_8.copy(), df_szn.copy(), 2, 2, True, ["season", "game_number"], False)
df_16vSZN = get_form_quick(df_16.copy(), df_szn.copy(), 2, 2, True, ["season", "game_number"], False)

df_2v8 = get_form_quick(df_2.copy(), df_8.copy(), 2, 8, False, ["season", "game_number"], False)
df_2v16 = get_form_quick(df_2.copy(), df_16.copy(), 2, 16, False, ["season", "game_number"], False)
df_2v32 = get_form_quick(df_2.copy(), df_32.copy(), 2, 32, False, ["season", "game_number"], False)
df_4v16 = get_form_quick(df_4.copy(), df_16.copy(), 4, 16, False, ["season", "game_number"], False)
df_4v32 = get_form_quick(df_4.copy(), df_32.copy(), 4, 32, False, ["season", "game_number"], False)
df_8v16 = get_form_quick(df_8.copy(), df_16.copy(), 8, 16, False, ["season", "game_number"], False)
df_8v32 = get_form_quick(df_8.copy(), df_32.copy(), 8, 32, False, ["season", "game_number"], False)

df_2v32

,game_id,team,opp_team,next_game_id,game_date,game_time,situation,season,game_number,score,...,scoreAdjustedShotsAttempts,unblockedShotAttempts,scoreAdjustedUnblockedShotAttempts,dZoneGiveaways,xGoalsFromxReboundsOfShots,xGoalsFromActualReboundsOfShots,reboundxGoals,totalShotCredit,scoreAdjustedTotalShotCredit,scoreFlurryAdjustedTotalShotCredit
0,2014-10-08BOSPHI,BOS,PHI,2014-10-09DETBOS,2014-10-08,19:30:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-10-08BOSPHI,PHI,BOS,2014-10-09PHINJD,2014-10-08,19:30:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014-10-08CGYVAN,CGY,VAN,2014-10-09EDMCGY,2014-10-08,22:00:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014-10-08CGYVAN,VAN,CGY,2014-10-11VANEDM,2014-10-08,22:00:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014-10-08LAKSJS,LAK,SJS,2014-10-11ARILAK,2014-10-08,22:00:00,5on5,2014,1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21541,2023-04-13WSHNJD,WSH,NJD,NaN,2023-04-13,19:00:00,5on5,2022,82,-0.03125,...,-1.977094,1.90625,2.707312,-1.53125,-0.020531,-0.241313,-0.263656,0.014125,0.056000,0.085312
21542,2023-04-14CBJBUF,BUF,CBJ,NaN,2023-04-14,19:30:00,5on5,2022,82,1.06250,...,12.177750,9.43750,10.174281,-1.81250,0.126437,0.100656,0.099094,0.982219,1.021812,0.990281
21543,2023-04-14CBJBUF,CBJ,BUF,NaN,2023-04-14,19:30:00,5on5,2022,82,-0.18750,...,4.419844,7.87500,6.608906,-1.21875,0.126031,0.332906,0.332906,0.879563,0.754281,0.743719
21544,2023-04-14NSHCOL,COL,NSH,NaN,2023-04-14,20:00:00,5on5,2022,82,0.00000,...,-4.224469,-2.68750,-2.237750,2.21875,-0.022219,-0.085500,-0.085500,-0.059938,-0.027625,-0.028563


In [20]:
def drop_nulls(df):
    df["next_game_id"].fillna("NA", inplace=True)
    return df.dropna()
    
    
df_2vSZN = drop_nulls(df_2vSZN.copy())
df_4vSZN = drop_nulls(df_4vSZN.copy())
df_8vSZN = drop_nulls(df_8vSZN.copy())
df_16vSZN = drop_nulls(df_16vSZN.copy())
df_32vSZN = drop_nulls(df_32vSZN.copy())

df_2v8 = drop_nulls(df_2v8.copy())
df_2v16 = drop_nulls(df_2v16.copy())
df_2v32 = drop_nulls(df_2v32.copy())
df_4v16 = drop_nulls(df_4v16.copy())
df_4v32 = drop_nulls(df_4v32.copy())
df_8v16 = drop_nulls(df_8v16.copy())
df_8v32 = drop_nulls(df_8v32.copy())


,game_id,team,opp_team,next_game_id,game_date,game_time,situation,season,game_number,score,...,scoreAdjustedShotsAttempts,unblockedShotAttempts,scoreAdjustedUnblockedShotAttempts,dZoneGiveaways,xGoalsFromxReboundsOfShots,xGoalsFromActualReboundsOfShots,reboundxGoals,totalShotCredit,scoreAdjustedTotalShotCredit,scoreFlurryAdjustedTotalShotCredit
404,2014-11-08BUFPIT,BUF,PIT,2014-11-11STLBUF,2014-11-08,19:00:00,5on5,2014,16,0.000000,...,-3.552714e-15,0.000000,3.552714e-15,0.000000,0.000000e+00,-2.775558e-17,0.000000e+00,2.220446e-16,0.000000e+00,2.220446e-16
410,2014-11-08FLACGY,CGY,FLA,2014-11-10CARCGY,2014-11-08,15:00:00,5on5,2014,16,0.000000,...,0.000000e+00,0.000000,0.000000e+00,0.000000,5.551115e-17,0.000000e+00,-2.775558e-17,0.000000e+00,0.000000e+00,2.220446e-16
418,2014-11-08PHICOL,COL,PHI,2014-11-11NYICOL,2014-11-08,19:00:00,5on5,2014,16,0.000000,...,-7.105427e-15,0.000000,0.000000e+00,0.000000,5.551115e-17,-5.551115e-17,-5.551115e-17,4.440892e-16,2.220446e-16,-2.220446e-16
426,2014-11-09ANAVAN,ANA,VAN,2014-11-12ANALAK,2014-11-09,21:00:00,5on5,2014,16,0.000000,...,1.421085e-14,0.000000,0.000000e+00,0.000000,5.551115e-17,0.000000e+00,0.000000e+00,2.220446e-16,0.000000e+00,2.220446e-16
427,2014-11-09ANAVAN,VAN,ANA,2014-11-11VANOTT,2014-11-09,21:00:00,5on5,2014,16,0.000000,...,7.105427e-15,0.000000,-1.065814e-14,0.000000,-1.110223e-16,0.000000e+00,0.000000e+00,0.000000e+00,2.220446e-16,-4.440892e-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21541,2023-04-13WSHNJD,WSH,NJD,NA,2023-04-13,19:00:00,5on5,2022,82,0.140244,...,-2.822962e+00,-0.830793,-1.407649e+00,1.199695,-1.175915e-02,5.536280e-02,4.664329e-02,-9.875152e-02,-1.321387e-01,-1.304893e-01
21542,2023-04-14CBJBUF,BUF,CBJ,NA,2023-04-14,19:30:00,5on5,2022,82,-0.234756,...,9.936616e-01,2.051829,2.266663e+00,0.144817,1.982317e-02,2.287165e-01,2.281067e-01,1.154970e-01,1.319314e-01,1.284070e-01
21543,2023-04-14CBJBUF,CBJ,BUF,NA,2023-04-14,19:30:00,5on5,2022,82,-0.172256,...,-1.643218e+00,-1.350610,-1.625694e+00,0.103659,-1.321341e-02,-1.176372e-02,-1.250762e-02,2.023018e-02,1.187043e-02,1.611738e-02
21544,2023-04-14NSHCOL,COL,NSH,NA,2023-04-14,20:00:00,5on5,2022,82,0.585366,...,2.213683e+00,0.827744,1.699572e+00,0.067073,2.134451e-02,4.049695e-02,3.855793e-02,2.065030e-01,2.616662e-01,2.573384e-01


In [32]:
def export(df, num):
    df = df[df.game_number != num]
    df.to_csv(CSV_DB_PATH + f'{season}/COMPARE_{num}vSZN_{situation}_{season}.csv', header=True, index=False)
    return df.copy()

df_2vSZN=export(df_2vSZN, 2)
df_4vSZN=export(df_4vSZN, 4)
df_8vSZN=export(df_8vSZN, 8)
df_16vSZN=export(df_16vSZN, 16)
df_32vSZN=export(df_32vSZN, 32)

df_8vSZN


,game_id,team,opp_team,next_game_id,game_date,game_time,situation,season,game_number,score,...,scoreAdjustedShotsAttempts,unblockedShotAttempts,scoreAdjustedUnblockedShotAttempts,dZoneGiveaways,xGoalsFromxReboundsOfShots,xGoalsFromActualReboundsOfShots,reboundxGoals,totalShotCredit,scoreAdjustedTotalShotCredit,scoreFlurryAdjustedTotalShotCredit
184,2014-10-23BOSNYI,BOS,NYI,2014-10-25TORBOS,2014-10-23,19:00:00,5on5,2014,9,0.055556,...,-0.589389,-0.527778,-0.573472,0.375000,0.002514,-0.014639,-0.005194,0.072653,0.076861,0.074625
187,2014-10-23CGYCAR,CGY,CAR,2014-10-25CGYWSH,2014-10-23,21:00:00,5on5,2014,9,0.097222,...,0.217042,-0.152778,0.122736,-0.152778,-0.000500,-0.005986,-0.010208,0.017903,0.032444,0.031889
212,2014-10-25CGYWSH,CGY,WSH,2014-10-28CGYMTL,2014-10-25,22:00:00,5on5,2014,10,-0.225000,...,0.082075,-0.350000,-0.407950,-0.425000,-0.005125,0.010725,0.006925,-0.050675,-0.055400,-0.054875
215,2014-10-25MINTBL,TBL,MIN,2014-10-28TBLARI,2014-10-25,20:00:00,5on5,2014,9,0.000000,...,0.089694,-0.069444,0.086750,0.416667,-0.002444,0.005194,0.009431,-0.035097,-0.021083,-0.018431
226,2014-10-25SJSBUF,BUF,SJS,2014-10-28TORBUF,2014-10-25,16:00:00,5on5,2014,9,0.027778,...,-0.063514,-0.555556,-0.493819,0.194444,-0.010361,0.027097,0.027097,-0.115931,-0.102597,-0.101375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21541,2023-04-13WSHNJD,WSH,NJD,NA,2023-04-13,19:00:00,5on5,2022,82,-0.734756,...,-5.449774,-2.518293,-3.086149,1.137195,-0.037009,0.106863,0.098143,-0.194689,-0.229951,-0.240552
21542,2023-04-14CBJBUF,BUF,CBJ,NA,2023-04-14,19:30:00,5on5,2022,82,0.390244,...,3.487287,4.051829,4.841851,-0.042683,0.049948,0.270966,0.270357,0.363122,0.429369,0.407845
21543,2023-04-14CBJBUF,CBJ,BUF,NA,2023-04-14,19:30:00,5on5,2022,82,-0.734756,...,-4.477155,-2.600610,-3.257506,0.228659,-0.016838,-0.049451,-0.050195,0.023793,-0.027942,-0.019445
21544,2023-04-14NSHCOL,COL,NSH,NA,2023-04-14,20:00:00,5on5,2022,82,0.710366,...,0.435808,-0.984756,0.406759,0.692073,0.002220,0.068997,0.067058,0.098753,0.189604,0.186838


In [33]:
## EXPORT
df_2v8.to_csv(CSV_DB_PATH + f'{season}/COMPARE_2v8_{situation}_{season}.csv', header=True, index=False)
df_2v16.to_csv(CSV_DB_PATH + f'{season}/COMPARE_2v16_{situation}_{season}.csv', header=True, index=False)
df_2v32.to_csv(CSV_DB_PATH + f'{season}/COMPARE_2v32_{situation}_{season}.csv', header=True, index=False)
df_4v16.to_csv(CSV_DB_PATH + f'{season}/COMPARE_4v16_{situation}_{season}.csv', header=True, index=False)
df_4v32.to_csv(CSV_DB_PATH + f'{season}/COMPARE_4v32_{situation}_{season}.csv', header=True, index=False)
df_8v16.to_csv(CSV_DB_PATH + f'{season}/COMPARE_8v16_{situation}_{season}.csv', header=True, index=False)
df_8v32.to_csv(CSV_DB_PATH + f'{season}/COMPARE_8v32_{situation}_{season}.csv', header=True, index=False)

#os.remove(CSV_TEMP_PATH + f'ROLLING_{num_games}_{situation}_{season}.csv')